#### Get y_list 

In [3]:
# Autoreload 
%load_ext autoreload
%autoreload 2

from pathlib import Path
from phosphosite.dataset import psp_filtered
psp_filtered

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,uniprot_id,mod_rsd,mod,site,res,pos
0,P31946,T2-p,p,T2,T,2
1,P31946,S6-p,p,S6,S,6
2,P31946,Y21-p,p,Y21,Y,21
3,P31946,T32-p,p,T32,T,32
4,P31946,S39-p,p,S39,S,39
...,...,...,...,...,...,...
239815,Q8IYH5,S474-p,p,S474,S,474
239816,Q8IYH5,S606-p,p,S606,S,606
239817,Q8IYH5,Y670-p,p,Y670,Y,670
239818,Q8IYH5,S677-p,p,S677,S,677


In [ ]:
# TODO: 
# have a process that filters down this list to only phosphosites that MAP ON 
# to the uniprot_id (i.e. same localised Serine as in the sequence) 
# use the same uniprot_id sequences as the embeddings and whatever the alphafold structures
# will have. 

In [8]:
from phosphosite.dataset.psp import load_psp_alternate
psp = load_psp_alternate(
    include_columns=["uniprot_id", "res", "pos", ]
)
psp

,uniprot_id,res,pos
0,P31946,T,2
1,P31946,S,6
2,P31946,Y,21
3,P31946,T,32
4,P31946,S,39
...,...,...,...
239815,Q8IYH5,S,474
239816,Q8IYH5,S,606
239817,Q8IYH5,Y,670
239818,Q8IYH5,S,677


In [9]:
# Order by uniprot_id 
psp = psp.sort_values(by=["uniprot_id", "res", "pos"])
psp

,uniprot_id,res,pos
26255,A0A024R4G9,S,16
26256,A0A024R4G9,S,20
26254,A0A024R4G9,T,14
156759,A0A075B759,S,40
156761,A0A075B759,T,93
...,...,...,...
111988,Q9Y6Y9,Y,131
43374,Q9Y6Z7,S,141
43375,Q9Y6Z7,T,155
64576,Q9YNA8,S,465


In [10]:
# subtract 1 from pos to get index. 
psp["index"] = psp["pos"] - 1

In [11]:
# group by uniprot_id to get the list of phosphosites for each uniprot_id (positions)
psp = psp.groupby("uniprot_id").agg(list)
psp

,res,pos,index
uniprot_id,,,
A0A024R4G9,"[S, S, T]","[16, 20, 14]","[15, 19, 13]"
A0A075B759,"[S, T, T, Y]","[40, 93, 119, 79]","[39, 92, 118, 78]"
A0A087WUL8,"[S, S, S, S, S, S, S, S, S, S, S, S, S, S]","[364, 608, 852, 1096, 1340, 1584, 1828, 2072, ...","[363, 607, 851, 1095, 1339, 1583, 1827, 2071, ..."
A0A096LNH2,"[S, S, S]","[799, 801, 895]","[798, 800, 894]"
A0A096LP49,"[S, S, S]","[478, 519, 566]","[477, 518, 565]"
...,...,...,...
Q9Y6Y8,"[S, S, S, S, S, S, S, S, S, S, S, S, S, S, S, ...","[134, 136, 258, 602, 636, 712, 737, 740, 742, ...","[133, 135, 257, 601, 635, 711, 736, 739, 741, ..."
Q9Y6Y9,"[Y, Y]","[22, 131]","[21, 130]"
Q9Y6Z7,"[S, T]","[141, 155]","[140, 154]"


#### Save unique protein ids for structure download.

In [ ]:
psp_uniprot_ids_to_use = psp.index.tolist()

with open("data/psp_uniprot_ids_to_use.txt", "w") as f:
    f.write("\n".join(psp_uniprot_ids_to_use))

In [12]:
# turn dataframe into a dictionary
site_dict = psp.to_dict(orient="index", )
site_dict

{'A0A024R4G9': {'res': ['S', 'S', 'T'],
  'pos': [16, 20, 14],
  'index': [15, 19, 13]},
 'A0A075B759': {'res': ['S', 'T', 'T', 'Y'],
  'pos': [40, 93, 119, 79],
  'index': [39, 92, 118, 78]},
 'A0A087WUL8': {'res': ['S',
   'S',
   'S',
   'S',
   'S',
   'S',
   'S',
   'S',
   'S',
   'S',
   'S',
   'S',
   'S',
   'S'],
  'pos': [364,
   608,
   852,
   1096,
   1340,
   1584,
   1828,
   2072,
   2316,
   2560,
   2804,
   3048,
   3292,
   3536],
  'index': [363,
   607,
   851,
   1095,
   1339,
   1583,
   1827,
   2071,
   2315,
   2559,
   2803,
   3047,
   3291,
   3535]},
 'A0A096LNH2': {'res': ['S', 'S', 'S'],
  'pos': [799, 801, 895],
  'index': [798, 800, 894]},
 'A0A096LP49': {'res': ['S', 'S', 'S'],
  'pos': [478, 519, 566],
  'index': [477, 518, 565]},
 'A0A096LP55': {'res': ['T'], 'pos': [11], 'index': [10]},
 'A0A0A0MR41': {'res': ['T'], 'pos': [616], 'index': [615]},
 'A0A0B4J2A2': {'res': ['S', 'S', 'S', 'S', 'T', 'Y'],
  'pos': [40, 95, 99, 110, 119, 79],
  'ind

In [13]:
position_dict = dict(zip(psp.index, psp["index"]))
position_dict 

{'A0A024R4G9': [15, 19, 13],
 'A0A075B759': [39, 92, 118, 78],
 'A0A087WUL8': [363,
  607,
  851,
  1095,
  1339,
  1583,
  1827,
  2071,
  2315,
  2559,
  2803,
  3047,
  3291,
  3535],
 'A0A096LNH2': [798, 800, 894],
 'A0A096LP49': [477, 518, 565],
 'A0A096LP55': [10],
 'A0A0A0MR41': [615],
 'A0A0B4J2A2': [39, 94, 98, 109, 118, 78],
 'A0A0B4J2F2': [574],
 'A0A0C4DFX5': [8],
 'A0A0J9YWL9': [30,
  373,
  375,
  377,
  436,
  438,
  440,
  457,
  459,
  461,
  604,
  606,
  608,
  625,
  629,
  686,
  690,
  707,
  709,
  711,
  728,
  732,
  852,
  854,
  856,
  35,
  680,
  722,
  34],
 'A0A0U1RQK7': [134, 122, 140],
 'A0A0U1RR07': [1023,
  1040,
  1071,
  1207,
  1211,
  1287,
  1319,
  1370,
  1408,
  1523,
  1561,
  1585,
  1615,
  1739,
  1099,
  1663,
  889,
  890,
  1515],
 'A0A1B0GTR3': [59, 60],
 'A0A5F9ZHR4': [330, 334],
 'A0AUZ9': [104, 204, 463, 525, 713, 821, 796, 791],
 'A0AV02': [98, 487, 661, 664, 468, 484, 106],
 'A0AV96': [44,
  395,
  507,
  539,
  41,
  518,
  532,


In [14]:
# Negative examples
import torch
indexes_dict = {} 
for key, pos_idx in position_dict.items():
    
    pos_idx = torch.tensor(pos_idx)
    pos_y = torch.ones(pos_idx.shape[0])
    
    neg_idx = [1,2,3]

    neg_idx = torch.tensor(neg_idx)
    neg_y = torch.zeros(neg_idx.shape[0])

    indexes_dict[key] = dict(
        idx=torch.cat([pos_idx, neg_idx]),
        y=torch.cat([pos_y, neg_y])
    )
    

In [78]:
indexes_dict

{'A0A024R4G9': {'idx': tensor([15, 19, 13,  1,  2,  3]),
  'y': tensor([1., 1., 1., 0., 0., 0.])},
 'A0A075B759': {'idx': tensor([ 39,  92, 118,  78,   1,   2,   3]),
  'y': tensor([1., 1., 1., 1., 0., 0., 0.])},
 'A0A087WUL8': {'idx': tensor([ 363,  607,  851, 1095, 1339, 1583, 1827, 2071, 2315, 2559, 2803, 3047,
          3291, 3535,    1,    2,    3]),
  'y': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.])},
 'A0A096LNH2': {'idx': tensor([798, 800, 894,   1,   2,   3]),
  'y': tensor([1., 1., 1., 0., 0., 0.])},
 'A0A096LP49': {'idx': tensor([477, 518, 565,   1,   2,   3]),
  'y': tensor([1., 1., 1., 0., 0., 0.])},
 'A0A096LP55': {'idx': tensor([10,  1,  2,  3]),
  'y': tensor([1., 0., 0., 0.])},
 'A0A0A0MR41': {'idx': tensor([615,   1,   2,   3]),
  'y': tensor([1., 0., 0., 0.])},
 'A0A0B4J2A2': {'idx': tensor([ 39,  94,  98, 109, 118,  78,   1,   2,   3]),
  'y': tensor([1., 1., 1., 1., 1., 1., 0., 0., 0.])},
 'A0A0B4J2F2': {'idx': tensor([574,   1,   2

`idx` is a tensor of indexes for the nodes that should be evaluated by the model.  
`y` is a tensor corresponding to `idx` of the true values that should be output for each of the nodes.

### Testing tensor indexing

In [79]:
import torch 
x = torch.tensor([0, 1, 2, 3, 4, 5])
indexes = torch.tensor([0, 1, 2, 4])
x

tensor([0, 1, 2, 3, 4, 5])

In [80]:
x[indexes]

tensor([0, 1, 2, 4])

In [81]:
out_of_order_indexes = torch.tensor([4, 2, 1, 0, 5, 5, 2])
x[out_of_order_indexes]

tensor([4, 2, 1, 0, 5, 5, 2])

#### Get all downloaded uniprot IDs to create dataset

In [82]:
from phosphosite import PHOSPHOSITE_PREDICT_DIR
root_dir = PHOSPHOSITE_PREDICT_DIR / "protein_graph_dataset"


import glob 
extension = "pdb"  # "pt"
which_dir = "raw" # "processed"
query = str(root_dir / which_dir / "*.{extension}".format(extension=extension))
downloaded_fns = glob.glob(query)
uniprot_ids = [Path(a).stem for a in downloaded_fns]
len(uniprot_ids)

17223

#### Load dataset with `y_list` included

In [84]:
from phosphosite.ml.dataset import PhosphoGraphDataset

uniprot_ids = uniprot_ids[0:10]
ds = PhosphoGraphDataset(
    root=root_dir,
    uniprot_ids=uniprot_ids, 
    y_label_map=indexes_dict,
)

Processing...
Processing O75807: 100%|██████████| 10/10 [00:04<00:00,  2.44it/s]

Successfully generated graphs for 10 out of 10 proteins.



Done!


In [85]:
ds.data

/home/cim/anaconda3/envs/phosphosite_ml/lib/python3.9/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Data(x=[7095, 1024], edge_index=[2, 2549], node_id=[10], b_factor=[7095], name=[10], num_nodes=7095)

In [86]:
ds.uniprot_ids

['Q9BX59',
 'Q14493',
 'P35498',
 'Q6PJF5',
 'Q8NEE6',
 'P26010',
 'Q92692',
 'O94818',
 'Q8WVI7',
 'O75807']

### Alternate dataset modified from `compot` 

This dataset is an alternate to the above `InMemoryDataset`-inheriting class.  


This `PhosphositeGraphDataset` class loads from disk and adds the `y` and `y_index` attributes to the data object which corresponds to the binary labels (phosphosite or unsite), and the indexes of nodes that have these labels; respectively. 

#### Setup

Load config objects etc.

In [88]:
from graphein.ml import ProteinGraphDataset
from pathlib import Path
import torch

from tqdm import tqdm

In [89]:
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.edges.distance import add_distance_threshold

from functools import partial

long_interaction_threshold = 5 # seq positions 
edge_threshold_distance = 6.0 # Å
new_edge_funcs = {"edge_construction_functions": [
    partial(
    add_distance_threshold, long_interaction_threshold=long_interaction_threshold, threshold=edge_threshold_distance)
]}
config = ProteinGraphConfig(
    granularity="CA",
    **new_edge_funcs,
)

from graphein.ml.conversion import GraphFormatConvertor

columns = [
    "b_factor",
    "name",
    "edge_index",
    "x", # T5 per-residue embedding
]
convertor = GraphFormatConvertor(
    src_format="nx", dst_format="pyg", verbose="gnn",
    columns=columns,
)

# List of functions that consume a nx.Graph and return a nx.Graph. Applied to graphs after construction but before conversion to pyg
from phosphosite.graphs.pyg import add_per_residue_embedding
graph_transforms = [
    add_per_residue_embedding,
]

# TODO: copy across code that adds the residue embedding from `compot`. 

In [90]:
from phosphosite import PHOSPHOSITE_PREDICT_DIR
root_dir =  PHOSPHOSITE_PREDICT_DIR / "protein_graph_dataset"

In [91]:
all_ids = list(psp.index)
processed_filenames = [Path(a).stem for a in glob.glob(str(root_dir / "processed" / "*.pt"))]
downloaded_filenames = [Path(a).stem for a in glob.glob(str(root_dir / "raw" / "*.pdb"))]


uniprot_ids = [a for a in all_ids if a in processed_filenames]
len(all_ids), len(downloaded_filenames), len(processed_filenames), len(uniprot_ids)

(17585, 17223, 17067, 17063)

In [92]:
uniprot_ids_downloaded = [a for a in all_ids if a in downloaded_filenames]
len(uniprot_ids_downloaded)

17222

In [93]:
for u in uniprot_ids_downloaded: 
    d = indexes_dict[u]
    continue

In [94]:
root_dir 

PosixPath('/home/cim/STRUCTURAL_MOTIFS/phosphosite/notebooks/phosphosite_prediction/protein_graph_dataset')

In [95]:
with open(root_dir / "valid_uniprot_ids.txt", "r") as f:
    valid_uniprot_ids = f.read().splitlines()

not_valid = [a for a in uniprot_ids_downloaded if a not in valid_uniprot_ids]
len(not_valid), len(uniprot_ids_downloaded)

(17173, 17222)

In [96]:
len(processed_filenames)

17067

In [97]:
not_yet_processed = [a for a in uniprot_ids_downloaded if a not in processed_filenames]
len(not_yet_processed)

159

In [115]:
from phosphosite.ml.graph_dataset import PhosphositeGraphDataset 
kwargs = dict(
    root=root_dir,
    graphein_config=config, 
    graph_transformation_funcs=graph_transforms,
    graph_format_convertor=convertor,
    pre_transform=None, # before saved to disk , after PyG conversion 
    pre_filter=None,    # whether it will be in final dataset
)

# Attempt to load the graphs that failed.
start = 50
n = 100
ds = PhosphositeGraphDataset(
    uniprot_ids=not_yet_processed[0:5],
    #uniprot_ids=not_valid[start:start+n],
    y_label_map=indexes_dict,
    **kwargs,
)
ds

Processing...
  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:03<00:00,  3.04s/it]

failed to convert graph None
failed to convert graph None
failed to convert graph None
failed to convert graph None
failed to convert graph None



Done!


PhosphositeGraphDataset()

In [116]:
# Load in the actual dataset (i.e. processed filenames)
ds = PhosphositeGraphDataset(
    uniprot_ids=processed_filenames,
    y_label_map=indexes_dict,
    **kwargs,
)

In [120]:
ds[0].y_index

tensor([ 345,  429,  430,  719,  789,  884,  899,  900,  917,  930, 1219,  340,
         344,  396,  400,  516,  974, 1123, 1126, 1135,    1,    2,    3])

#### Number of failed graphs

In [122]:
processed_filenames = [Path(a).stem for a in glob.glob(str(root_dir / "processed" / "*.pt"))]
not_yet_processed = [a for a in uniprot_ids_downloaded if a not in processed_filenames]
len(not_yet_processed)

159

In [123]:
len(processed_filenames), ds

(17067, PhosphositeGraphDataset(17067))

### Construct dataloaders

In [124]:
from phosphosite.ml import get_dataloader_split
train_loader, valid_loader, test_loader = get_dataloader_split(ds, batch_size=8, train_batch_size=8)
len(train_loader), len(valid_loader), len(test_loader)

(1365, 342, 427)

In [125]:
batch = next(iter(train_loader))
batch

DataBatch(x=[3729, 1024], edge_index=[2, 2145], node_id=[8], b_factor=[3729], name=[8], num_nodes=3729, y_index=[101], y=[101], batch=[3729], ptr=[9])

In [126]:
batch.y_index

tensor([  59,   64,   67,   69,   72,   74,   84,   88,  213,   63,  211,    1,
           2,    3,  341,  257,  258,  259,  549,  704,  804,  813,  548,  552,
         354,  355,  356, 1284, 1287, 1288, 1290, 1360,  860,  861,  862, 1391,
        1470, 1492, 2101, 2150, 1490, 1567, 2093, 1468, 1795, 1914, 1917, 1368,
        1369, 1370, 2292, 2293, 2303, 2308, 2312, 2353, 2404, 2409, 2503, 2569,
        2334, 2339, 2349, 2532, 2278, 2301, 2400, 2437, 2523, 2242, 2243, 2244,
        2591, 2722, 2740, 2747, 2923, 2743, 2957, 2734, 2951, 3003, 2585, 2586,
        2587, 3154, 3160, 3256, 3298, 3301, 3670, 3699, 3147, 3277, 3146, 3348,
        3475, 3477, 3037, 3038, 3039])

In [127]:
batch.y

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0.,
        1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.])

In [128]:
# index batch.y_index where batch.y == 1 
batch.y_index[batch.y == 0]

tensor([   1,    2,    3,  257,  258,  259,  354,  355,  356,  860,  861,  862,
        1368, 1369, 1370, 2242, 2243, 2244, 2585, 2586, 2587, 3037, 3038, 3039])

### Test batch on model forward pass

In [135]:
from phosphosite.model import PhosphoGAT

model = PhosphoGAT()
model.eval()

batch = next(iter(train_loader))
y_hat = model(batch)
y = batch.y 

In [136]:
y_hat, y

(tensor([[0.4437],
         [0.4596],
         [0.4875],
         ...,
         [0.4799],
         [0.4776],
         [0.4670]], grad_fn=<SigmoidBackward0>),
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1.,
         1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         0., 0., 0.]))

In [130]:
y.shape

torch.Size([147])

In [131]:
# turn y_hat into int
y_hat = torch.argmax(y_hat, dim=1)
y_hat.shape

torch.Size([5514])

In [134]:
y_hat.sum(), y.sum()

(tensor(0), tensor(123.))